In [1]:
import os
import sys
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
from IPython.display import Audio
import matplotlib.pyplot as plt
from tensorflow.python.platform import gfile
from tensorflow.python.summary.writer.writer import FileWriter
import matplotlib.pyplot as plt
import wuw.data
import wuw.models
import wuw.utils
import yaml
import json
from tqdm.notebook import trange, tqdm
from PIL import Image

tf.compat.v1.disable_eager_execution()

plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100

2023-06-05 12:58:35.433511: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 12:58:36.137497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
conf_fn = "conf/lstm-s.yaml"
expdir = "exp/utt3s_trainset3_lstm_step10k_allbg5"
logdir = expdir + "/logs"

with open(conf_fn, "r") as stream:
    try:
        conf_yaml = yaml.safe_load(stream)
        # print(json.dumps(conf_yaml, indent=2))
    except yaml.YAMLError as exc:
        print(exc)

In [3]:
tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.InteractiveSession()
model = wuw.models.select_model(
    conf_yaml["data_conf"], conf_yaml["feat_conf"], conf_yaml["model_conf"]
)

conf_yaml["model_conf"] = model.prepare_model_settings()

2023-06-05 12:58:37.212442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 12:58:37.234627: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 12:58:37.234867: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
wav_data = tf.compat.v1.placeholder(
    tf.float32,
    [conf_yaml["model_conf"]["desired_samples"], 1],
    name="decoded_sample_data",
)

spectrogram = tf.raw_ops.AudioSpectrogram(
    input=wav_data,
    window_size=conf_yaml["model_conf"]["window_size_samples"],
    stride=conf_yaml["model_conf"]["window_stride_samples"],
    magnitude_squared=True,
)

mfcc_ = tf.raw_ops.Mfcc(
    spectrogram=spectrogram,
    sample_rate=conf_yaml["model_conf"]["sample_rate"],
    upper_frequency_limit=conf_yaml["feat_conf"]["upper_frequency_limit"],
    lower_frequency_limit=conf_yaml["feat_conf"]["lower_frequency_limit"],
    filterbank_channel_count=conf_yaml["feat_conf"]["filterbank_channel_count"],
    dct_coefficient_count=conf_yaml["feat_conf"]["dct_coefficient_count"],
)

fingerprint_frequency_size = conf_yaml["model_conf"]["dct_coefficient_count"]
fingerprint_time_size = conf_yaml["model_conf"]["spectrogram_length"]
fingerprint_input = tf.reshape(
    mfcc_, [-1, fingerprint_time_size * fingerprint_frequency_size]
)


logits, _ = model.forward(
    fingerprint_input, conf_yaml["model_conf"]["model_size_info"], is_training=False
)
tf.sigmoid(logits, name="labels_sigmoid")
model.load_variables_from_checkpoint(
    sess, tf.compat.v1.train.latest_checkpoint(expdir + "/best")
)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from exp/utt3s_trainset3_lstm_step10k_allbg5/best/lstm_9837.ckpt-9798


/disk2/projects/ResearchProjects/KWSProjects/train_hiFPToi/wuw/models.py:299: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(lstm_units,
2023-06-05 12:58:37.864595: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 12:58:37.864756: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bu

In [5]:
frozen_graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(
    sess, sess.graph_def, ["labels_sigmoid"]
)
pb_path = os.path.join(expdir, conf_yaml["model_conf"]["name"] + ".pb")
tf.compat.v1.train.write_graph(
    frozen_graph_def, os.path.dirname(pb_path), os.path.basename(pb_path), as_text=False
)
print(f"Saved frozen graph to {pb_path}")

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Saved frozen graph to exp/utt3s_trainset3_lstm_step10k_allbg5/lstm.pb


In [6]:
def model_split(
    pb_fn, fe_fn, core_fn, full_fn, input_arrays, split_arrays, outout_arrays
):
    converter_fe = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
        pb_fn, input_arrays=input_arrays, output_arrays=split_arrays
    )
    converter_fe.allow_custom_ops = True
    converter_fe.optimizations = [tf.lite.Optimize.DEFAULT]

    tflite_fe = converter_fe.convert()
    with open(fe_fn, "wb") as f:
        f.write(tflite_fe)

    converter_core_model = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
        pb_fn, input_arrays=split_arrays, output_arrays=outout_arrays
    )
    converter_core_model.allow_custom_ops = True
    converter_core_model.optimizations = [tf.lite.Optimize.DEFAULT]

    tflite_core_model = converter_core_model.convert()
    with open(core_fn, "wb") as f:
        f.write(tflite_core_model)

    converter_core_model = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
        pb_fn, input_arrays=input_arrays, output_arrays=outout_arrays
    )
    converter_core_model.allow_custom_ops = True
    converter_core_model.optimizations = [tf.lite.Optimize.DEFAULT]

    tflite_core_model = converter_core_model.convert()
    with open(full_fn, "wb") as f:
        f.write(tflite_core_model)

In [7]:
model_split(
    pb_path,
    expdir + "/" + conf_yaml["model_conf"]["name"] + "_fe.tflite",
    expdir + "/" + conf_yaml["model_conf"]["name"] + "_core.tflite",
    expdir + "/" + conf_yaml["model_conf"]["name"] + ".tflite",
    ["decoded_sample_data"],
    ["Mfcc"],
    ["labels_sigmoid"],
)

2023-06-05 12:58:38.003567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 12:58:38.004161: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 12:58:38.004327: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf